### Integrantes:
- José Antonio Miranda Baños

- Fernando Josué Matute Soto

- Paolo Alfonso Reyes Ramírez

- Ricardo Navarro Gómez

In [19]:
# Import necesary libraries
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import fuzzylite as fl

# Uncomment and run the following command if FuzzyLite is not installed yet
import warnings
warnings.filterwarnings('ignore')

## Truck Problem using Mandani

In [20]:
#Declaring and Initializing the Fuzzy Engine

engine = fl.Engine(name='Truck',
    description='Truck Parking Problem',)

#Defining the Input Variables (Fuzzification)
engine.input_variables = [
    fl.InputVariable(
    name='Distance', description='Distance from parking', enabled=True,
    minimum=0, maximum=20.01, lock_range=False,
    terms=[fl.Gaussian('lefter', 0, 2),
           fl.Gaussian('left',5, 2),
           fl.Gaussian('center',10, 2),
           fl.Gaussian('right',15,2),
           fl.Gaussian('righter',20,2)]
    ),
    fl.InputVariable(
    name='Angle', description="Angle from the parking spot", enabled=True,
    minimum=-90, maximum=270.01, lock_range=False,
    terms=[fl.Gaussian('l_horizontal',180, 20),
           fl.Gaussian('l_diagonal',135, 20),
           fl.Gaussian('perpendicular',90, 20),
           fl.Gaussian('r_diagonal',45, 20),
           fl.Gaussian('r_horizontal',0, 20)])
]

#Defining the Output Variables (Defuzzification)
engine.output_variables = [
    fl.OutputVariable(
    name='Steering_angle', description='Steering angle needed for parking', enabled=True,
    minimum=-40, maximum=40.01, lock_range=False,
    aggregation=fl.Maximum(), defuzzifier=fl.Centroid(200), default_value = fl.nan,
    lock_previous=False,
    terms=[fl.Gaussian('LEFT', 35, 5),
            fl.Gaussian('left_2', 26.1, 5),
            fl.Gaussian('left_3', 17.2, 5),
            fl.Gaussian('left_4', 8.3, 5),
            fl.Gaussian('center', 0, 5),
            fl.Gaussian('right_2', -12.3, 5),
            fl.Gaussian('right_3', -21.2, 5),
            fl.Gaussian('right_4', -28.1, 5),
            fl.Gaussian('RIGHT', -35, 5)])]

Defining the Output Variables (Defuzzification) for Mamdani inference system.

### Rules
1. If lefter AND r_horizontal, THEN RIGht
2. If lefter AND r_diagonal, THEN center
3. If lefter AND perpendicular, THEN leFT

4. If left AND r_horizontal, THEN RIGHt
5. If left AND r_diagonal, THEN RIght
6. If left AND perpendicular, leFT

7. If center AND r_horizontal, THEN RIGHT
8. If center AND r_diagonal, THEN RIGHt
9. If center AND perpendicular, THEN center


In [21]:
#Creation of Fuzzy Rule Base (Mamdani)
# Antecedent terms serving as inputs to this rule.
engine.rule_blocks = [
      fl.RuleBlock(
      name='Truck Parking', description='Truck FIS for Steering_angle to Parking Spot',
      enabled=True,
      conjunction=fl.Minimum(), # AND
      disjunction=fl.Maximum(), # OR
      implication=fl.Minimum(),
      activation=fl.General(), # Activation method that activates every rule of a rule block in insertion order
      rules=[
        fl.Rule.create("if Distance is lefter and Angle is r_horizontal then Steering_angle is right_3", engine),
        fl.Rule.create("if Distance is lefter and Angle is r_diagonal then Steering_angle is center", engine),
        fl.Rule.create("if Distance is lefter and Angle is perpendicular then Steering_angle is left_4", engine),
        fl.Rule.create("if Distance is left and Angle is r_horizontal then Steering_angle is right_4", engine),
        fl.Rule.create("if Distance is left and Angle is r_diagonal then Steering_angle is right_2", engine),
        fl.Rule.create("if Distance is left and Angle is perpendicular then Steering_angle is left_4", engine),
        fl.Rule.create("if Distance is center and Angle is r_horizontal then Steering_angle is RIGHT", engine),
        fl.Rule.create("if Distance is center and Angle is r_diagonal then Steering_angle is right_4", engine),
        fl.Rule.create("if Distance is center and Angle is perpendicular then Steering_angle is center", engine)]
      )
]

## TSK
The weight-based defuzzifier is computed using the results of the membership functions as values and the activation degrees as weights

## Tsukamoto

The weight-based defuzzifier uses the activation degrees as weights, and for the values uses the arguments of the membership functions that produce the respective activation degrees.

In [ ]:
# Let's add a Tsukamoto output block to the FIS
parking = fl.OutputVariable()
parking.name = ""
parking.description = "Steering Angle for Parking"
parking.enabled = True
parking.range = (0.000, 30.000)
parking.lock_range = False # lock the value to the range of the variable
parking.aggregation = None
parking.defuzzifier = fl.WeightedAverage("Tsukamoto")
parking.default_value = fl.nan
parking.lock_previous = False # lock the previous value of the output variable
parking.terms.append(fl.Concave("cheap", 5.0, 4.0)) # inflection, end
parking.terms.append(fl.SShape("average", 12.0, 20.0)) # start, end
parking.terms.append(fl.Sigmoid("generous", 20.0, 30)) # inflection, slope
engine.output_variables.append(parking)

In [ ]:
# Reglas Tsukamoto
tsukamoto = fl.RuleBlock()
tsukamoto.name = "Tsukamoto"
tsukamoto.description = "Tsukamoto inference"
tsukamoto.enabled = True
tsukamoto.conjunction = fl.Minimum()
tsukamoto.disjunction = fl.Maximum()
tsukamoto.implication = fl.Minimum()
tsukamoto.activation = fl.General()
tsukamoto.rules=[
        fl.Rule.create("if Distance is lefter and Angle is r_horizontal then Steering_angle is right_3", engine),
        fl.Rule.create("if Distance is lefter and Angle is r_diagonal then Steering_angle is center", engine),
        fl.Rule.create("if Distance is lefter and Angle is perpendicular then Steering_angle is left_4", engine),
        fl.Rule.create("if Distance is left and Angle is r_horizontal then Steering_angle is right_4", engine),
        fl.Rule.create("if Distance is left and Angle is r_diagonal then Steering_angle is right_2", engine),
        fl.Rule.create("if Distance is left and Angle is perpendicular then Steering_angle is left_4", engine),
        fl.Rule.create("if Distance is center and Angle is r_horizontal then Steering_angle is RIGHT", engine),
        fl.Rule.create("if Distance is center and Angle is r_diagonal then Steering_angle is right_4", engine),
        fl.Rule.create("if Distance is center and Angle is perpendicular then Steering_angle is center", engine)]

engine.rule_blocks.append(tsukamoto)

In [178]:
data = np.array([[  0.  ,   1.  ,   0.  , -19.  ],
       [  1.  ,   1.95,   9.37, -17.95],
       [  2.  ,   2.88,  18.23, -16.9 ],
       [  3.  ,   3.79,  26.59, -15.85],
       [  4.  ,   4.65,  34.44, -14.8 ],
       [  5.  ,   5.45,  41.78, -13.75],
       [  6.  ,   6.18,  48.6 , -12.7 ],
       [  7.  ,   7.48,  54.91, -11.65],
       [  8.  ,   7.99,  60.71, -10.6 ],
       [  9.  ,   8.72,  65.99,  -9.55],
       [ 10.  ,   9.01,  70.75,  -8.5 ],
       [ 11.  ,   9.28,  74.98,  -7.45],
       [ 12.  ,   9.46,  78.7 ,  -6.4 ],
       [ 13.  ,   9.59,  81.9 ,  -5.34],
       [ 14.  ,   9.72,  84.57,  -4.3 ],
       [ 15.  ,   9.81,  86.72,  -3.25],
       [ 16.  ,   9.88,  88.34,  -2.2 ],
       [ 17.  ,   9.91,  89.44,   0.  ]])
data

array([[  0.  ,   1.  ,   0.  , -19.  ],
       [  1.  ,   1.95,   9.37, -17.95],
       [  2.  ,   2.88,  18.23, -16.9 ],
       [  3.  ,   3.79,  26.59, -15.85],
       [  4.  ,   4.65,  34.44, -14.8 ],
       [  5.  ,   5.45,  41.78, -13.75],
       [  6.  ,   6.18,  48.6 , -12.7 ],
       [  7.  ,   7.48,  54.91, -11.65],
       [  8.  ,   7.99,  60.71, -10.6 ],
       [  9.  ,   8.72,  65.99,  -9.55],
       [ 10.  ,   9.01,  70.75,  -8.5 ],
       [ 11.  ,   9.28,  74.98,  -7.45],
       [ 12.  ,   9.46,  78.7 ,  -6.4 ],
       [ 13.  ,   9.59,  81.9 ,  -5.34],
       [ 14.  ,   9.72,  84.57,  -4.3 ],
       [ 15.  ,   9.81,  86.72,  -3.25],
       [ 16.  ,   9.88,  88.34,  -2.2 ],
       [ 17.  ,   9.91,  89.44,   0.  ]])

In [179]:
real_value_list = [dato[-1] for dato in data]
print(real_value_list)
predicted_value_list=[]

[-19.0, -17.95, -16.9, -15.85, -14.8, -13.75, -12.7, -11.65, -10.6, -9.55, -8.5, -7.45, -6.4, -5.34, -4.3, -3.25, -2.2, 0.0]


In [180]:
for dato in data:
  theta_fis.inputs({'distance': dato[1], 'angle': dato[2]})
  theta_fis.compute()
  predicted_value_list.append(theta_fis.output['steering_angle'])

for i, predicted_value in enumerate(predicted_value_list):
 print(f'Valor original {real_value_list[i]}, Predict: {predicted_value} ')

Valor original -19.0, Predict: -19.30151712269368 
Valor original -17.95, Predict: -17.121630855473132 
Valor original -16.9, Predict: -15.639000608841577 
Valor original -15.85, Predict: -15.894118560386962 
Valor original -14.8, Predict: -14.392872424716181 
Valor original -13.75, Predict: -11.483022072077402 
Valor original -12.7, Predict: -9.65410176768104 
Valor original -11.65, Predict: -10.121545620356832 
Valor original -10.6, Predict: -8.42997314896901 
Valor original -9.55, Predict: -9.87919288467768 
Valor original -8.5, Predict: -8.66291428635454 
Valor original -7.45, Predict: -7.390830734733301 
Valor original -6.4, Predict: -6.0748239102437545 
Valor original -5.34, Predict: -4.915701803185293 
Valor original -4.3, Predict: -4.0273978305695755 
Valor original -3.25, Predict: -3.325989062171009 
Valor original -2.2, Predict: -2.8292737044465635 
Valor original 0.0, Predict: -2.483240903382649 


In [181]:
def get_RMSE(real_values_list, predicted_value_list):
        return(np.square(np.subtract(real_values_list, predicted_value_list)).mean())

In [182]:
error = get_RMSE(real_value_list, predicted_value_list)
print(error)

1.7257520340177384
